In [7]:
import easyocr
import re
import cv2
import numpy as np

In [8]:
reader = easyocr.Reader(['en'], gpu = True)  # с ru хуже

In [9]:
def adjust_gamma(image, gamma=1.2):
    # build a lookup table mapping the pixel values [0, 255] to
    # their adjusted gamma values
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")

    # apply gamma correction using the lookup table
    return cv2.LUT(image, table)

# Automatic brightness and contrast optimization with optional histogram clipping
def automatic_brightness_and_contrast(image, clip_hist_percent=1):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Calculate grayscale histogram
    hist = cv2.calcHist([gray],[0],None,[256],[0,256])
    hist_size = len(hist)
    
    # Calculate cumulative distribution from the histogram
    accumulator = []
    accumulator.append(float(hist[0]))
    for index in range(1, hist_size):
        accumulator.append(accumulator[index -1] + float(hist[index]))
    
    # Locate points to clip
    maximum = accumulator[-1]
    clip_hist_percent *= (maximum/100.0)
    clip_hist_percent /= 2.0
    
    # Locate left cut
    minimum_gray = 0
    while accumulator[minimum_gray] < clip_hist_percent:
        minimum_gray += 1
    
    # Locate right cut
    maximum_gray = hist_size -1
    while accumulator[maximum_gray] >= (maximum - clip_hist_percent):
        maximum_gray -= 1
    
    # Calculate alpha and beta values
    alpha = 255 / (maximum_gray - minimum_gray)
    beta = -minimum_gray * alpha  
    auto_result = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    return (auto_result, alpha, beta)

def clahe(frame):

    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    lab[:,:,0] = clahe.apply(lab[:,:,0])

    # Converting image from LAB Color model to BGR color space
    image_clahe = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    return image_clahe

In [10]:
def image_preparation(frame):

    # minmal size rectangle, px   
    frame_for_auto = frame
    frame_for_auto = cv2.fastNlMeansDenoisingColored(frame_for_auto ,None, 2, 2, 4, 4)
    frame_for_auto = clahe(frame_for_auto)
    frame_for_auto = adjust_gamma(frame_for_auto)        
    
    frame_auto, alpha, betta = automatic_brightness_and_contrast(frame_for_auto, clip_hist_percent=1)

    out_frame = frame_auto
    return out_frame

def get_rects(frame):
    minSize = int(frame.shape[0]/2.1)
    return reader.readtext(frame, min_size = minSize, width_ths=0.1)

In [11]:
def run_processing(filename, out_filename='video'):
    cap = cv2.VideoCapture(filename)

    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fourcc = cv2.VideoWriter_fourcc(*'X264')    
    # fourcc = cv2.VideoWriter_fourcc('a','v','c','1')  
    out=cv2.VideoWriter(f'{out_filename}.mkv',
                        fourcc,
                        24, (frame_width, frame_height), True)

    ind = 0
    # errInd = 49

    newVagonCnt = 0
    # если newVagonCnt > newVagonCntMax, то номер, вероятно, уже не в кадре
    newVagonCntMax = 1


    numberCandidate = []

    totalNumbers = []

    if not cap.isOpened():
        print("Error opening video file")
    else:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            ind += 1
            # if ind < 10:
            #     continue
            # if ind > 28:
            #     break
            
            # if ind < 0:
            #     continue
            # if ind > 130:
            #     break

            frame_for_rect = frame
            frame_for_rect = image_preparation(frame)
            # print(frame_for_rect.dtype, frame_for_rect.shape)

            rects = get_rects(frame_for_rect) # OCR function

            # if ind > errInd:
            #     print(ind, rects)

            newVagonCntShouldInc = True

            # print(ind)

            if len(rects) > 0:
                for elem in rects:

                    

                    # if ind > errInd:
                    if type(elem[0][0][0]) == float:
                        print("elem:", elem)
                        print(np.array(elem).astype(int))
                    if type(elem[0][2][0]) == float:
                        print("elem:", elem)
                        print(np.array(elem).astype(int))

                    number_str = re.sub("[^0-9]", "", elem[1].replace(" ", ""))

                    if len(number_str) < 6:
                        
                        continue

                    

                    elem[0][0] = np.array(elem[0][0]).astype(int)
                    elem[0][2] = np.array(elem[0][2]).astype(int)

                    colorRect = (0, 0, 255)

                    # if len(number_str) < 8:
                        # по идее здесь нужно сохранять кадр

                    
                    # if len(number_str) == 8:
                    #     newVagonCntShouldInc = False
                    #     newVagonCnt = 0
                    #     colorRect = (0, 255, 0)

                    #     shouldAdd = True
                    #     for cand in numberCandidate:
                    #         if cand[0] == number_str:
                    #             cand[1] += 1
                    #             shouldAdd = False
                    #     if shouldAdd:
                    #         numberCandidate.append([number_str, 1])
                    #         print(numberCandidate)
                    #         print(elem)

                    if len(number_str) == 8:
                        colorRect = (0, 255, 0)

                    newVagonCntShouldInc = False
                    newVagonCnt = 0

                    shouldAdd = True
                    for cand in numberCandidate:
                        if cand[0] == number_str:
                            cand[1] += 1
                            shouldAdd = False
                    if shouldAdd:
                        numberCandidate.append([number_str, 1])
                        print(numberCandidate)
                        print(elem)

                    # если символов меньше или больше 8, то нужно добавлять в датасет для обучения
                    # дополнительно можно проверять верность полученного номера (по принципу его формирования)                    

                    frame = cv2.rectangle(frame, elem[0][0], elem[0][2], colorRect, 2, )
                    frame = cv2.putText(frame, number_str,
                                        elem[0][0], fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1.2,
                                            color=colorRect, thickness=2)
            
            if newVagonCntShouldInc:

                if newVagonCnt > newVagonCntMax:
                    print(numberCandidate)
                    if len(numberCandidate):
                        condFind = []
                        # массив со строками из 8 символов
                        numberCandidateEightCnt = []
                        condMax = -1

                        # набор массива строк из 8 символов
                        for cond in numberCandidate:
                            if len(cond[0]) == 8:
                                numberCandidateEightCnt.append(cond)

                        # если массив ненулевой, то скорее всего номер здесь, а все остальные строки не нужны
                        if len(numberCandidateEightCnt):
                            numberCandidate = numberCandidateEightCnt

                        # определение наиболее повторяемого номера
                        for cond in numberCandidate:
                            print("cond")
                            print(cond)
                            if cond[1] > condMax:
                                condMax = cond[1]
                                condFind = cond
                                
                                print(condFind)
                        totalNumbers.append(condFind[0])
                        numberCandidate = []
                else:
                    newVagonCnt += 1
            
            print(ind)

            # np.hstack((frame, frame_for_rect))
            # image_clahe = np.hstack((frame, frame_for_rect))
            cv2.imshow(filename, frame)

            out.write(frame)
            key = cv2.waitKey(1)
            if key == 27: # Esc нажать для остановки видео
                break
            if key == ord(' '):
                key = cv2.waitKey(0)
            if key == ord('p'):
                key = cv2.waitKey(0)

    print("end write")
    cap.release()
    out.release()
    # print(totalNumbers)
    cv2.destroyAllWindows()

    return totalNumbers

In [12]:
# run_processing('Sample/vagons.mp4', (640, 480), out_filename='processed_vagons')
tmp = run_processing('Sample/cisterns.avi', out_filename='processed_vagons')
# print(tmp)
totalNumbers = tmp
print(totalNumbers)
etalon = [57001133,50524875,50833102,50865385,50833128,50833060,50865864,50865906,50865732,50865435,50865633,50832971,50865666,50832930]
# etalon = np.array([57001133,50524875,50833102,50865385,50833128,50833060,50865864,50865906,50865732,50865435,50865633,50832971,50865666,50832930]).astype(str)

result = []
trueCnt = 0
# result = totalNumbers == etalon.astype(str)
for a,b in zip(totalNumbers, etalon):
    resComp = (a == str(b))
    if resComp:
        trueCnt+=1
    result.append(resComp)

print(result)

print(trueCnt/len(result))



[['5700133', 1]]
([array([127, 127]), [298, 127], array([298, 181]), [127, 181]], '57001i33', 0.5274932199704536)
1
[['5700133', 1], ['570033', 1]]
([array([ 90, 127]), [264, 127], array([264, 182]), [90, 182]], '5700ii33', 0.37302713527191467)
2
[['5700133', 1], ['570033', 1], ['57001133', 1]]
([array([ 56, 129]), [228, 129], array([228, 184]), [56, 184]], '57001133', 0.5366004039208053)
3
4
5
6
7
[['5700133', 2], ['570033', 1], ['57001133', 2]]
cond
['57001133', 2]
['57001133', 2]
8
[]
9
[]
10
[]
11
[]
12
[]
13
[]
14
[]
15
[]
16
[]
17
[]
18
[]
19
[]
20
[]
21
[]
22
[]
23
[]
24
[]
25
[]
26
[]
27
[]
28
[]
29
[]
30
[]
31
[]
32
[]
33
[]
34
[]
35
[]
36
[]
37
[]
38
[]
39
[]
40
[]
41
[['5052481', 1]]
([array([216, 191]), [352, 191], array([352, 247]), [216, 247]], '5052481', 0.6656270744542602)
42
[['5052481', 1], ['50524873', 1]]
([array([198, 194]), [352, 194], array([352, 247]), [198, 247]], '50524873', 0.2619578915322694)
43
[['5052481', 1], ['50524873', 1], ['50524875', 1]]
([array([162

In [13]:
# totalNumbers = np.array(['57001133', '50524875', '50873102'])

etalon = np.array([57001133,50524875,50833102,50865385,50833128,50833060,50865864,50865906,50865732,50865435,50865633,50832971,50865666,50832930]).astype(str)

# print(totalNumbers)
print (etalon.astype(str))

# result = []
# # result = totalNumbers == etalon.astype(str)
# for a,b in zip(totalNumbers, etalon):
#     result.append(a==b)

# print(result)

['57001133' '50524875' '50833102' '50865385' '50833128' '50833060'
 '50865864' '50865906' '50865732' '50865435' '50865633' '50832971'
 '50865666' '50832930']


In [26]:
np.savetxt("totalNum.txt", np.array(totalNumbers).astype(np.uint32), fmt='%i')